In [1]:
import models.lm as lm
import models.prompt as prompt
import models.gpt_gen as gpt_gen
import data
import defs

import os
import json

%load_ext autoreload
%autoreload 2

# IC examples rendering

In [2]:
ic_examples = [
    {
        "name": "recolor_y",
        "nl_description": "Recolor all objects to color Y"
    },
    {
        "name": "extend_to_max",
        "nl_description": "Extend all the objects of size 1 toward the object of maximum size, without overlapping."
    },
    {
        "name": "three_move_recolor",
        "nl_description": "Move all objects of size 3 one step up, and then recolor them to color B."
    },
    {
        "name": "move_to_grey",
        "nl_description": "Move all objects of color different than X, toward the object of color X, if they are neighbors of it."
    }
]

ic_ex_pmpt = ""

for i, ic_example in enumerate(ic_examples):
    # print(f"## DEMONSTRATION TASK {i + 1}\n")
    ic_ex_pmpt += f"## DEMONSTRATION TASK {i + 1}\n\n"
    ic_example_name = ic_example["name"]
    nl_description = ic_example["nl_description"]

    desc = data.task_description(f"ic_examples/{ic_example_name}.json", print_test=False, color_map="char")
    # print("### INPUT")
    # print(desc)
    ic_ex_pmpt += f"### INPUT\n{desc}\n"

    dsl_src_path = f"dsl/v0_3/ic_examples/{ic_example_name}.dsl"
    with open(dsl_src_path, "r") as f:
        dsl_src = f.read()
    example_response = json.dumps({
        "nl_description": nl_description,
        "code": dsl_src
    }, indent=4)

    # print("### EXPECTED OUTPUT")
    # print(example_response)
    # print("\n")
    ic_ex_pmpt += f"### EXPECTED OUTPUT\n{example_response}\n\n\n"

ic_ex_pmpt = ic_ex_pmpt[:-2]
print(ic_ex_pmpt)

## DEMONSTRATION TASK 1

### INPUT
PAIR 1
INPUT GRID:
O O O O O O O O
O O O O O R O O
O R O O O R O R
O R R O O R O O
O O O O O O O O
O R R O O O O O
O R R O R R O O
O O O O O O O O
OUTPUT GRID:
O O O O O O O O
O O O O O Y O O
O Y O O O Y O Y
O Y Y O O Y O O
O O O O O O O O
O Y Y O O O O O
O Y Y O Y Y O O
O O O O O O O O

PAIR 2
INPUT GRID:
O O O O O O O O O O
O O O O R O O O O O
O G O O R O O W O O
O O G O O O W W O O
O O G O O O O O O O
O O O O X O O O O O
O O O O O O F F F O
O B O O O O O O O O
O B B B B O O C C O
O O O O O O O C O O
OUTPUT GRID:
O O O O O O O O O O
O O O O Y O O O O O
O Y O O Y O O Y O O
O O Y O O O Y Y O O
O O Y O O O O O O O
O O O O Y O O O O O
O O O O O O Y Y Y O
O Y O O O O O O O O
O Y Y Y Y O O Y Y O
O O O O O O O Y O O

### EXPECTED OUTPUT
{
    "nl_description": "Recolor all objects to color Y",
    "code": "(do\n    (rule\n        (vars (this))\n        (filter)\n        (apply\n            (update_color Y)\n        )\n    )\n)"
}


## DEMONSTRATION TASK 2


# Main prompt

In [3]:
### SYSTEM

system_prompt = """You are an assistant chatbot with human-like perception, reasoning and learning capabilities.
You can solve tasks concisely, efficiently, and moreover, correctly.
Let's engage in perception- and logic-based tasks.
"""

system_prompt = system_prompt + """You only output source code.
No explanations or any other text.
Only code.
"""





### PREAMBLE
preamble_simple = """You are an efficient assistant for logical reasoning and code generation.
You will help me solve a visual perception and reasoning task.
I will first provide you with the definition of a Domain Specific Language you will use for writing a solution for the task.
I will then provide you with some examples of how to use the DSL.
I will then present you with the description of the task that you will be tested in.
You will then respond the queries I make regarding the solution of the task.
"""

preamble_simple_no_ex = """You are an efficient assistant for logical reasoning and code generation.
You will help me solve a visual perception and reasoning task.
I will first provide you with the definition of a Domain Specific Language you will use for writing a solution for the task.
I will then present you with the description of the task that you will be tested in.
You will then respond the queries I make regarding the solution of the task.
"""

preamble_nl_ex = """
You are a efficient assistant for code generation.
The setup for this interaction is as follows:
I will first show you the EBNF grammar definition for a domain specific language (DSL) that you will use to write source code in.
Then, I will show you a small number of example programs in this DSL.
Then, I will present to you a visual perception and reasoning task, and you will be asked to write a program in the DSL that solves this task.
"""






### DSL
dsl_preamble = """This is the definition of the DSL you will use to solve the task.
It is given as a context-free grammar in the EBNF format used by the Lark parser generator, with some informative comments about the semantics.
You will return a string that is parseable by the `program` non-terminal of the grammar.
"""

dsl_path = "dsl/v0_3/dsl.lark"
with open(dsl_path, "r") as f:
    dsl_grammar = f.read()

dsl_prompt = f"{dsl_preamble}\n```\n{dsl_grammar}\n```\n"





# ### EXAMPLES

# nl_examples = """I will now show you some example programs in the DSL, along with natural language description of their behavior.
# """

# # ic_examples = """I will now show you some example tasks along with their solutions in the DSL.
# # They might be simpler than your test task, but they should allow you to understand the behavior of the DSL.
# # """

### IC EXAMPLES

ic_examples_path = "models/templates/v0_3/ic_examples.txt"
with open(ic_examples_path, "r") as f:
    ic_examples_prompt = f.read()

ic_examples_preamble = """Now I will show you some demonstration tasks along with the output you would be expected to produce for each of them.
"""

# ic_examples = f"{ic_examples_preamble}\n{ic_examples_prompt}"
ic_examples = f"{ic_examples_preamble}\n{ic_ex_pmpt}"

### TASK
task_preamble = """Now we continue with the visual perception and reasoning task.
The input for the task is a small number of pairs of grids of characters.
The value of each of the cells of the grids are the colors defined in the DSL, so we can think of grids as images.
Each pair of images correspond to an input-output example for an unknown program P.
For each pair, the program P is evaluated on the image grid and operates on the objects that appear in it.
The output of the program is then the output image.
The objects in the images are easy and natural to identify for humans, so there is no need to define them explicitly.
However you are able to abstract them correctly, and the DSL is interpreted with the same correct abstraction.
"""

test_task_preamble = """Now follows task you will be evaluated on.
Output the solution as a JSON object, which should contain both a natural language description of the solution and the solution written in the DSL.
The code should be parseable by the DSL grammar.
The JSON must have the following structure:

{
    "nl_description": "TO_BE_FILLED",
    "code": "TO_BE_FILLED"
}

## TEST TASK"""

### QUERY

# query_code_only = """
# Write a program in the DSL that will solve this task.
# The program should be enclosed in backticks as per Markdown syntax.
# """


In [4]:
def make_task_prompt(task_id):
    task_description = data.task_description(task_id=task_id, print_test=False, color_map="char")
    task_prompt = f"{test_task_preamble}\n\n{task_description}"

    prompt = "\n\n".join([
        preamble_simple_no_ex,
        dsl_prompt,
        task_preamble,
        ic_examples,
        task_prompt
    ])

    return prompt

# sample_task_id = "ae3edfdc"
sample_task_id = "ddf7fa4f"
prompt = make_task_prompt(sample_task_id)
print(prompt)

You are an efficient assistant for logical reasoning and code generation.
You will help me solve a visual perception and reasoning task.
I will first provide you with the definition of a Domain Specific Language you will use for writing a solution for the task.
I will then present you with the description of the task that you will be tested in.
You will then respond the queries I make regarding the solution of the task.


This is the definition of the DSL you will use to solve the task.
It is given as a context-free grammar in the EBNF format used by the Lark parser generator, with some informative comments about the semantics.
You will return a string that is parseable by the `program` non-terminal of the grammar.

```
library: "(" program* ")"

// Rules are executed one after another, in the order they appear.
// There could be no rules, in which case the program does nothing.
program: "(" "do" rule* ")"

// First, it declares the variables that will be used in the filter and transfo

In [5]:
def query_task(task_id, n_responses, output_dir, model):
    """
    Returns a list of n_responses responses to the prompt
    """
    pmpt = make_task_prompt(task_id=task_id)
    lm_gateway = lm.LanguageModel(model=model)
    system_prompt_path = os.path.join(defs.PROJECT_ROOT, "models/templates/system.txt")
    with open(system_prompt_path, "r") as f:
        system_prompt = f.read()
    response = lm_gateway.query(pmpt, n_responses, system_prompt=system_prompt, log=True)
    result = gpt_gen.process_responses_json(response)
    result["task_id"] = task_id

    # save the completions to files
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with open(os.path.join(output_dir, f"{task_id}.json"), "w") as f:
        json.dump(result, f, indent=4)
    return result

def query_task_list(task_ids, n_responses, model):
    # create a directory for the output
    timestamp = defs.get_timestamp(micros=False)
    output_dir = os.path.join(defs.PROJECT_ROOT, "models/logs", f"gens_{timestamp}")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # query each task
    for task_id in task_ids:
        print(f"Getting generations for task {task_id}")
        query_task(task_id, n_responses, output_dir, model)

In [10]:
# model = "gpt-3.5-turbo-0125"
# model = "gpt-4-0125-preview"
# model = "gpt-4-turbo"
# model = "gpt-4o"
# model = "gpt-4o-mini"
model = "gpt-4o-2024-05-13"

# n_responses = 150
# n_responses = 128
n_responses = 100
# n_responses = 75
# n_responses = 50
# n_responses = 30
# n_responses = 25
# n_responses = 20
# n_responses = 15
# n_responses = 10
# n_responses = 5

# sample_task_id = "ae3edfdc"
# task_list = [sample_task_id]

# task_list_path = "dsl/v0_3/example_candidates.txt"
task_list_path = "dsl/v0_3/arga_subset.txt"
# task_list_path = "dsl/v0_3/missing.txt"
with open(task_list_path, "r") as f:
    task_list = [t_id for t_id in f.read().splitlines() if t_id]

query_task_list(task_list, n_responses, model)

Getting generations for task 00d62c1b
Request took 13.51 seconds
Getting generations for task 025d127b
Request took 7.18 seconds
Getting generations for task 05f2a901
Request took 8.14 seconds
Getting generations for task 08ed6ac7
Request took 32.93 seconds
Getting generations for task 0962bcdd
Request took 11.70 seconds
Getting generations for task 0ca9ddb6
Request took 13.74 seconds
Getting generations for task 0d3d703e
Request took 22.81 seconds
Getting generations for task 0e206a2e
Request took 13.57 seconds
Getting generations for task 150deff5
Request took 10.41 seconds
Getting generations for task 1a07d186
Request took 16.46 seconds
Getting generations for task 1b60fb0c
Request took 8.16 seconds
Getting generations for task 1caeab9d
Request took 8.87 seconds
Getting generations for task 1e0a9b12
Request took 12.52 seconds
Getting generations for task 1f0c79e5
Request took 8.13 seconds
Getting generations for task 2204b7a8
Request took 9.97 seconds
Getting generations for task 22

# Change format of old gens

In [26]:
from models.gpt_gen import is_parseable
from random import shuffle

gens_dir = "dsl/v0_3/generations/arga_gpt4o_m100_20240516"
out_dir = "dsl/v0_3/generations/gens_20240516_gpt-4o_m100_strict"

# list all files in the directory and get those that contain "valid.txt"
valid_files = [f for f in os.listdir(gens_dir) if "_valid.txt" in f]

# iterate over the files
for valid_file in valid_files:
# for valid_file in valid_files[:1]:
    # get the task id from the file name
    task_id = valid_file.split("_")[0]
    with open(os.path.join(gens_dir, valid_file), "r") as f:
        data = json.load(f)
    print(task_id)
    result = {"completions": []}
    for completion in data:
        completion = json.loads(completion)
        # parse, _ = is_parseable(completion["code"])
        # assert parse
        result["completions"].append(completion)
    
    shuffle(result["completions"])
    result["completions"] = result["completions"][:100]

    with open(os.path.join(out_dir, f"{task_id}.json"), "w") as f:
        json.dump(result, f, indent=4)

3906de3d
444801d8
5521c0d9
6855a6e4
6aa20dc0
d511f180
d687bc17
6150a2bd
68b16354
0962bcdd
b8cdaf2b
00d62c1b
3aa6fb7a
ae3edfdc
6d58a25d
ce22a75a
aabf363d
1f0c79e5
39e1d7f9
22168020
d2abd087
d23f8c26
db93a21d
32597951
85c4e7cd
f76d97a5
41e4d17e
b775ac94
025d127b
6d75e8bb
b2862040
d037b0a7
7f4411dc
6e82a1ae
3eda0437
74dd1130
57aa92db
4093f84a
952a094c
ed36ccf7
b527c5c6
9565186b
d89b689b
5c0a986e
e40b9e2f
ea32f347
99fa7670
e8dc4411
855e0971
67385a82
d90796e8
228f6490
bda2d7a6
6c434453
e26a3af2
913fb3ed
08ed6ac7
e21d9049
b548a754
67a423a3
2c608aff
c8f0f002
91714a58
272f95fa
83302e8f
9edfc990
67a3c6ac
a5313dff
98cf29f8
50cb2852
a9f96cdd
7e0986d6
0d3d703e
bb43febb
794b24be
5582e5ca
b60334d2
95990924
a699fb00
f8a8fe49
29c11459
e73095fd
63613498
22eb0ac0
7447852a
22233c11
a48eeaf7
4612dd53
543a7ed5
54d9e175
4347f46a
3c9b0459
25d487eb
c9f8e694
3bdb4ada
e76a88a6
31aa019c
b27ca6d3
d5d6de2d
9dfd6313
694f12f3
ddf7fa4f
88a10436
7ddcd7ec
6455b5f5
b7249182
1e0a9b12
1a07d186
25ff71a9
ba26e723
8d510a79
4

In [27]:
# list all the files in the out directory, make sure there are 160
# and each has at least 100 completions
out_files = os.listdir(out_dir)
assert len(out_files) == 160
for out_file in out_files:
    # print(out_file)
    with open(os.path.join(out_dir, out_file), "r") as f:
        data = json.load(f)
    assert len(data["completions"]) == 100